# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')


from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pickle

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
# load data from database
engine = create_engine('sqlite:///DiastersResponse.db')
df = pd.read_sql_table('Message', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

In [4]:
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_sample = df.sample(10)
df_sample

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
25408,29340,"Jakarta, Indonesia-Relief -- Green Helmets of ...",None,news,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
3963,4537,I forget the site next to me giving aid and ca...,Mwen bliye sit kote m ka dpoze cv an ou ka reb...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
12888,15374,Maybe those big red double decker sight seeing...,None,social,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
23656,27384,Indonesian Military requested (11/01/05) IOM t...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5197,5917,I would like to get messages on the 4636 servi...,Mwen tarenmen jwenn mesaj sevis 4636 la. svp,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9613,10828,I write you to know how i can find a job?,(Mwen ekri nou pou nka dim koman map jwenn yon...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19053,22256,He also condemned the recent police baton-char...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8247,9252,Please how much the earthquake pass this morni...,Tanpri a konbyen tremblement de terre a pase la,direct,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1
15968,18821,Hubei has launched a fund drive within the pro...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22441,26000,"Further to the text, the Council asked the Uni...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_sample['message']

25408    Jakarta, Indonesia-Relief -- Green Helmets of ...
3963     I forget the site next to me giving aid and ca...
12888    Maybe those big red double decker sight seeing...
23656    Indonesian Military requested (11/01/05) IOM t...
5197     I would like to get messages on the 4636 servi...
9613            I write you to know how i can find a job? 
19053    He also condemned the recent police baton-char...
8247     Please how much the earthquake pass this morni...
15968    Hubei has launched a fund drive within the pro...
22441    Further to the text, the Council asked the Uni...
Name: message, dtype: object

In [21]:
for index, row in df_sample.iterrows():
    print(row['message'])
    cates = []
    for i, v in row.items():
        if v == 1:
            cates.append(i)
    print(' ,'.join(cates))

Jakarta, Indonesia-Relief -- Green Helmets of Germany has completed construction of 62 wooden houses for tsunami survivors in village Pulau Kayu, subdistrict of Susoh, Southwest Aceh.
related ,aid_related ,shelter ,weather_related ,storm
I forget the site next to me giving aid and can give me a bed please 
related ,request ,aid_related ,shelter ,direct_report
Maybe those big red double decker sight seeing busses can volunteer to take people around? #Sandy #publictransitnightmareThe morning commute- walking over the Williamsburg Bridge #Sandy http://t.co/fGRlTCQr
related ,transport ,weather_related ,storm ,direct_report
Indonesian Military requested (11/01/05) IOM to provide 10 power generator sets.
related
I would like to get messages on the 4636 service,please. 

I write you to know how i can find a job? 

He also condemned the recent police baton-charge against protesters in Muzaffarabad when the government tried to evacuate them from their homes.

Please how much the earthquake pass

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    # Obtain tokens
    tokens = word_tokenize(text)
    
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Clean tokens to be return
    clean_tokens = []
    
    # Stop words 
    stop_words = set(stopwords.words('english'))
    
    # Loop through and return preprocess tokens
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        if clean_token not in stop_words:
            clean_tokens.append(clean_token)
    
    return clean_tokens

In [6]:
tokenize(X[0])

['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
# Build the first pipeline
first_pipeline = Pipeline([
     ('vect', CountVectorizer(tokenizer=tokenize)),
     ('tfidf', TfidfTransformer()),
     ('clf', MultiOutputClassifier(RandomForestClassifier())),
 ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split train-test data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
# Train the pipeline
first_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
# Predict the result
y_pred = first_pipeline.predict(X_test)

In [18]:
def display_report_on_each_output(y_pred, y_test):
    """ 
        A function that iterates all column and print out the metrics

    """
    for ith, column in enumerate(y_test.columns):
        print(column.upper(), 'accuracy: ', round(accuracy_score(y_test[column], y_pred[:,ith]), 2))
        print(classification_report(y_test[column], y_pred[:,ith]))

In [20]:
# Results
display_report_on_each_output(y_pred, y_test)

RELATED
              precision    recall  f1-score   support

           0       0.61      0.37      0.46      1523
           1       0.82      0.93      0.87      4979
           2       0.70      0.13      0.23        52

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.71      0.48      0.52      6554
weighted avg       0.77      0.79      0.77      6554

REQUEST
              precision    recall  f1-score   support

           0       0.89      0.98      0.94      5425
           1       0.84      0.43      0.57      1129

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.87      0.71      0.75      6554
weighted avg       0.88      0.89      0.87      6554

OFFER
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6396
           1       1.00      0.02      0.04       158

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.98      0.98      0.97      6554

MISSING_PEOPLE
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6482
           1       1.00      0.03      0.05        72

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.99      0.51      0.52      6554
weighted avg       0.99      0.99      0.98      6554

REFUGEES
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6309
           1       0.62      0.06      0.11       245

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.79      0.53      0.55      6554
weighted avg       0.95      0.96      0.95      6

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# Use GridSearch to find better parameters
parameters = {
    'vect__ngram_range': [(1,1), (1,2)],
    'vect__max_df': [1.0, 0.7],
    'clf__estimator__min_samples_leaf': [1, 3]
}

second_pipeline = Pipeline([
     ('vect', CountVectorizer(tokenizer=tokenize)),
     ('tfidf', TfidfTransformer()),
     ('clf', MultiOutputClassifier(RandomForestClassifier())),
 ])

second_pipeline_cv = GridSearchCV(second_pipeline, param_grid=parameters,  verbose=3, n_jobs=2)
second_pipeline_cv.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  24 out of  24 | elapsed: 29.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2)], 'vect__max_df': [1.0, 0.7], 'clf__estimator__min_samples_leaf': [1, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
# Results
y_pred = second_pipeline_cv.predict(X_test)
display_report_on_each_output(y_pred, y_test)

RELATED accuracy:  0.81
              precision    recall  f1-score   support

           0       0.61      0.43      0.50      1451
           1       0.84      0.92      0.88      5050
           2       0.50      0.09      0.16        53

   micro avg       0.81      0.81      0.81      6554
   macro avg       0.65      0.48      0.51      6554
weighted avg       0.79      0.81      0.79      6554

REQUEST accuracy:  0.88
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5425
           1       0.79      0.44      0.56      1129

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.84      0.71      0.75      6554
weighted avg       0.87      0.88      0.87      6554

OFFER accuracy:  1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

   micro avg       1.00      1.00      1.00      6554
   ma

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



MONEY accuracy:  0.97
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6371
           1       0.57      0.02      0.04       183

   micro avg       0.97      0.97      0.97      6554
   macro avg       0.77      0.51      0.51      6554
weighted avg       0.96      0.97      0.96      6554

MISSING_PEOPLE accuracy:  0.99
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6462
           1       0.67      0.02      0.04        92

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.83      0.51      0.52      6554
weighted avg       0.98      0.99      0.98      6554

REFUGEES accuracy:  0.97
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6332
           1       0.47      0.04      0.07       222

   micro avg       0.97      0.97      0.97      6554
   macro avg       0.72      0.52      0.53      

In [11]:
# Best parameters
cv.best_params_

{'clf__estimator__min_samples_leaf': 1,
 'vect__max_df': 0.7,
 'vect__ngram_range': (1, 1)}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
# Built the model using LightGBM library
third_pipeline =Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(lgb.LGBMClassifier())),
])

In [26]:
third_pipeline.fit(X_train, y_train)
y_pred = third_pipeline.predict(X_test)
display_report_on_each_output(y_pred, y_test)

RELATED accuracy:  0.83
              precision    recall  f1-score   support

           0       0.66      0.52      0.58      1451
           1       0.87      0.92      0.89      5050
           2       0.62      0.19      0.29        53

   micro avg       0.83      0.83      0.83      6554
   macro avg       0.72      0.54      0.59      6554
weighted avg       0.82      0.83      0.82      6554

REQUEST accuracy:  0.9
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      5425
           1       0.78      0.58      0.66      1129

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.85      0.77      0.80      6554
weighted avg       0.89      0.90      0.89      6554

OFFER accuracy:  1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

   micro avg       1.00      1.00      1.00      6554
   mac

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.96      0.98      0.97      5959
           1       0.77      0.57      0.66       595

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.86      0.78      0.81      6554
weighted avg       0.94      0.95      0.94      6554

CLOTHING accuracy:  0.99
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6441
           1       0.84      0.48      0.61       113

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.92      0.74      0.80      6554
weighted avg       0.99      0.99      0.99      6554

MONEY accuracy:  0.97
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6371
           1       0.62      0.22      0.33       183

   micro avg       0.97      0.97      0.97      6554
   macro avg       0.80      0.61      0.66      6554
weighted avg       0.97    

### 9. Export your model as a pickle file

In [27]:
with open('classifier.pkl', 'wb') as fp:
    pickle.dump(third_pipeline, fp)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.